<a href="https://colab.research.google.com/github/nimaghadirian/FRQI-Implementation/blob/main/Quantum_Image_Processing_FRQI%2C_NEQR%2C_and_QHED_in_Noiseless%2C_Noisy%2C_and_Error_Corrected_Scenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group: The Q-Brains**




# ***Here's a brief overview of what the code does:***

**Image Processing:** The code begins by loading an image, converting it to grayscale, and resizing it if desired. This preprocessing step is necessary as the subsequent quantum algorithms are designed to work with grayscale images.

**Quantum Circuit Creation:** Next, the code constructs quantum circuits for the FRQI (Flexible Representation of Quantum Image), NEQR (Novel Enhanced Quantum Representation), and QHED (Quantum Hadamard Edge Detection) methods. These circuits encode the grayscale values of the image into quantum states using specific quantum gates.

**Quantum Simulation:**

- ***Noiseless Simulation:*** The created quantum circuits are simulated using a quantum simulator without any noise. This scenario provides a baseline for the ideal behavior of the quantum image processing algorithms.

- ***Noisy Simulation with Error Correction (Bit-Flip Method):*** The quantum circuits are executed on the quantum simulator with a noise model that emulates the errors present in real quantum computers. Additionally, an error correction step is applied to the circuits using the Bit-Flip method to mitigate the effects of noise. This scenario demonstrates the performance of the quantum image processing algorithms when error correction techniques, specifically the Bit-Flip method, are employed.

- ***Noisy Simulation without Error Correction:*** The circuits are executed on the quantum simulator with a noise model that emulates the errors present in real quantum computers, but without applying any error correction techniques. This scenario allows for an analysis of the impact of noise on the quantum image processing algorithms without the benefit of error mitigation.

By simulating the circuits in **these three scenarios**, the code provides a comprehensive understanding of the behavior and performance of the quantum image processing algorithms under different conditions, including noiseless, noisy with error correction (using the Bit-Flip method), and noisy without error correction. This analysis helps assess the effectiveness of the Bit-Flip error correction technique in improving the accuracy and reliability of quantum computations in the presence of noise.

**Result Visualization:** Finally, the results of the quantum simulations are visualized using bar charts. These charts depict the measurement outcomes of the quantum circuits, allowing for a comparison between the noiseless and noisy simulations. The visualization helps demonstrate the impact of noise on the accuracy of the quantum image processing algorithms.

The objective of this code is to showcase the application of quantum computing in image processing tasks and provide a visual analysis of the outcomes under ideal and noisy conditions. The inclusion of error correction demonstrates the potential for error mitigation in quantum computations.



---



# ***Why Quantum Hadamard Edge Detection (QHED) and not QSobel ?***

In general, classical edge detection algorithms rely mostly on the computation of image gradients i.e. identifying locations in the image for dark-to-light (or light-to-dark) intensity transitions. Hence, the worst case time complexity for most of them is
. This means that each pixel needs to be processed individually to determine the gradients.

On the contrary, quantum edge detection algorithms like QSobel provide exponential speedup compared to the existing classical edge detection algorithms. However, there are some steps involved in the algorithm that make it quite inefficient, for example the COPY operation and a quantum black box to calculate the gradients of all the pixels. For both the operations, there is no single efficient implementation that is known as of now and is a complex topic of research. Hence, the need for a much more efficient algorithm is fulfilled by the Quantum Hadamard Edge Detection (QHED) algorithm.

https://learn.qiskit.org/course/ch-applications/quantum-edge-detection

In [ ]:
#Part 1: Importing necessary libraries and defining the function to process the image
import plotly.graph_objects as go
from qiskit import QuantumCircuit, Aer, execute
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import depolarizing_error
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# Define a function to process the image
def process_image(image_path):
    image = Image.open(image_path).convert('L') # Open the image and convert it to grayscale
    image_data = np.array(image.getdata(), dtype=np.uint8).reshape(image.size[::-1])  # Transform the image data into a Numpy array
    return image_data, image.size  # Return the image data and the original size of the image


#Part 2: Asking the user for image path and processing the image
image_path = input("Please enter the image path: ")
image_data, original_size = process_image(image_path)  # Process the image
print(f"The original size of the image is {original_size[0]} x {original_size[1]} pixels.")

# Ask user if they want to resize the image
resize_choice = input("Do you want to resize the image? (yes/no): ")
if resize_choice.lower() == 'yes':
    n = int(input("Enter the new dimension of the image (n x n): "))
    image_data = Image.fromarray(image_data).resize((n, n))
    image_data = np.array(image_data)
else:
    n = min(original_size)


#Part 3: Defining the FRQI, NEQR, and QHED functions
def frqi(n, image_data):
    qc = QuantumCircuit(n * 3, n * 3)  # Create a quantum circuit
    for i in range(n):
        qc.h(i)  # Apply Hadamard gate
        phase = image_data[i] / 255.0  # Calculate the phase
        qc.p(phase * np.pi, i)  # Apply phase gate
        qc.cx(i, i + n)  # Apply controlled-X gate
        qc.cx(i, i + 2 * n)  # Apply controlled-X gate
    qc.measure(range(n, 2 * n), range(n))  # Measure the qubits
    return qc

def neqr(n):
    qc = QuantumCircuit(n, n)  # Create a quantum circuit
    for i in range(n):
        qc.x(i)    # Apply X gate
    qc.measure(range(n), range(n))   # Measure the qubits
    return qc

def qhed(n, image_data):
    qc = QuantumCircuit(n * 3, n * 3)  # Create a quantum circuit
    for i in range(n):
        qc.h(i)   # Apply Hadamard gate
        phase = image_data[i] / 255.0  # Calculate the phase
        qc.p(phase * np.pi, i)  # Apply phase gate
        qc.cx(i, i + n)  # Apply controlled-X gate
        qc.cx(i, i + 2 * n)  # Apply controlled-X gate
    qc.barrier()  # Add a barrier for visual clarity
    qc.h(range(n, 2 * n))  # Apply Hadamard gate to the second half of the qubits
    qc.barrier()  # Add a barrier for visual clarity
    for i in range(n):
        qc.measure(i + n, i)  # Measure the qubits
    return qc


#Part 4: Creating FRQI, NEQR, and QHED circuits
image_data = image_data.flatten()
n = len(image_data)
frqi_circuit = frqi(n, image_data)
neqr_circuit = neqr(n)
qhed_circuit = qhed(n, image_data)


#Part 5: Defining a function to create a noise model with a specified error rate
def get_noise(p_error):
    error_gate1 = depolarizing_error(p_error, 1)
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, ['h', 'x'])
    return noise_model

noise_model = get_noise(0.01)



#Part 6: Defining a function to execute the circuit and get the results
def execute_circuit(circuit, noise_model=None, error_correction=False):
    simulator = Aer.get_backend('qasm_simulator')
    if error_correction:
        bit_flip_circuit = bit_flip_code(circuit.num_qubits, circuit)
        circuit = bit_flip_circuit
    result = execute(circuit, simulator, noise_model=noise_model, shots=1024).result()
    counts = result.get_counts()
    return counts


#Part 7: Defining a function to apply the bit flip code error correction
def bit_flip_code(n, circuit):
    qc = QuantumCircuit(n * 3, n * 3)
    for i in range(n):
        qc.cx(i, i + n)
        qc.cx(i, i + 2 * n)
    qc.barrier()
    qc = qc.compose(circuit)
    qc.barrier()
    for i in range(n):
        qc.cx(i, i + n)
        qc.cx(i, i + 2 * n)
        qc.ccx(i + n, i + 2 * n, i)
    return qc


#Part 8: Executing the circuits in different scenarios

frqi_counts_noiseless = execute_circuit(frqi_circuit) # Execute the FRQI circuit with no noise
neqr_counts_noiseless = execute_circuit(neqr_circuit) # Execute the NEQR circuit with no noise
qhed_counts_noiseless = execute_circuit(qhed_circuit) # Execute the QHED circuit with no noise
frqi_counts_noisy = execute_circuit(frqi_circuit, noise_model) # Execute the FRQI circuit with a noise model
neqr_counts_noisy = execute_circuit(neqr_circuit, noise_model) # Execute the NEQR circuit with a noise model
qhed_counts_noisy = execute_circuit(qhed_circuit, noise_model) # Execute the QHED circuit with a noise model

# Execute the FRQI circuit with a noise model and error correction
frqi_counts_noisy_error_correction = execute_circuit(frqi_circuit, noise_model, error_correction=True)

# Execute the NEQR circuit with a noise model and error correction
neqr_counts_noisy_error_correction = execute_circuit(neqr_circuit, noise_model, error_correction=True)

# Execute the QHED circuit with a noise model and error correction
qhed_counts_noisy_error_correction = execute_circuit(qhed_circuit, noise_model, error_correction=True)


#Part 9: Plotting the results (FRQI)
# Create a bar plot trace for noiseless FRQI results
frqi_trace_noiseless = go.Bar(
    x=list(frqi_counts_noiseless.keys()),    # X values are measurement outcomes
    y=list(frqi_counts_noiseless.values()),  # Y values are counts of each outcome
    name='Noiseless',  # Name of the trace
    marker=dict(color='rgba(31, 119, 180, 0.7)')  # Set color and opacity
)


# Create a bar plot trace for noisy FRQI results
frqi_trace_noisy = go.Bar(
    x=list(frqi_counts_noisy.keys()),
    y=list(frqi_counts_noisy.values()),
    name='Noisy',
    marker=dict(color='rgba(255, 127, 14, 0.7)')  # Adjust the opacity here
)

frqi_trace_noisy_error_correction = go.Bar(
    x=list(frqi_counts_noisy_error_correction.keys()),
    y=list(frqi_counts_noisy_error_correction.values()),
    name='Noisy with Error Correction',
    marker=dict(color='rgba(44, 160, 44, 0.7)')  # Adjust the opacity here
)

# Create a figure and add the traces
fig_frqi = go.Figure(data=[frqi_trace_noiseless, frqi_trace_noisy, frqi_trace_noisy_error_correction])
fig_frqi.update_layout(      # Update layout of the figure
    title='FRQI Results',    # Title of the plot
    xaxis_title='Measurement Outcome',  # X-axis label
    yaxis_title='Counts',    # Y-axis label
    barmode='group',   # Bars are grouped side by side
    bargap=0.2,  # Increase the bargap value for more space between bars
)
fig_frqi.show()  # Display the figure


#Part 10: Plotting the results (NEQR)
neqr_trace_noiseless = go.Bar(  # Create a bar plot trace for noiseless NEQR results
    x=list(neqr_counts_noiseless.keys()),    # X values are measurement outcomes
    y=list(neqr_counts_noiseless.values()),  # Y values are counts of each outcome
    name='Noiseless',  # Name of the trace
    marker=dict(color='rgba(31, 119, 180, 0.7)')  # Set color and opacity
)

# Create a bar plot trace for noisy NEQR results
neqr_trace_noisy = go.Bar(
    x=list(neqr_counts_noisy.keys()),
    y=list(neqr_counts_noisy.values()),
    name='Noisy',
    marker=dict(color='rgba(255, 127, 14, 0.7)')  # Set color and opacity
)

# Create a bar plot trace for noisy NEQR results with error correction
neqr_trace_noisy_error_correction = go.Bar(
    x=list(neqr_counts_noisy_error_correction.keys()),
    y=list(neqr_counts_noisy_error_correction.values()),
    name='Noisy with Error Correction',
    marker=dict(color='rgba(44, 160, 44, 0.7)')  # Set color and opacity
)

# Create a figure and add the traces
fig_neqr = go.Figure(data=[neqr_trace_noiseless, neqr_trace_noisy, neqr_trace_noisy_error_correction])
fig_neqr.update_layout(   # Update layout of the figure
    title='NEQR Results',  # Title of the plot
    xaxis_title='Measurement Outcome',  # X-axis label
    yaxis_title='Counts',    # Y-axis label
    width=800,   # Width of the plot
    height=400,  # Height of the plot
)
fig_neqr.show()   # Display the figure


#Part 11: Plotting the results (QHED)
qhed_trace_noiseless = go.Bar(  # Create a bar plot trace for noiseless QHED results
    x=list(qhed_counts_noiseless.keys()),    # X values are measurement outcomes
    y=list(qhed_counts_noiseless.values()),  # Y values are counts of each outcome
    name='Noiseless',  # Name of the trace
    marker=dict(color='rgba(31, 119, 180, 0.7)')  # Set color and opacity
)

# Create a bar plot trace for noisy QHED results
qhed_trace_noisy = go.Bar(
    x=list(qhed_counts_noisy.keys()),
    y=list(qhed_counts_noisy.values()),
    name='Noisy',
    marker=dict(color='rgba(255, 127, 14, 0.7)')  # Set color and opacity
)

# Create a bar plot trace for noisy QHED results with error correction
qhed_trace_noisy_error_correction = go.Bar(
    x=list(qhed_counts_noisy_error_correction.keys()),
    y=list(qhed_counts_noisy_error_correction.values()),
    name='Noisy with Error Correction',
    marker=dict(color='rgba(44, 160, 44, 0.7)')  # Set color and opacity
)


# Create a figure and add the traces
fig_qhed = go.Figure(data=[qhed_trace_noiseless, qhed_trace_noisy, qhed_trace_noisy_error_correction])
fig_qhed.update_layout(  # Update layout of the figure
    title='QHED Results',  # Title of the plot
    xaxis_title='Measurement Outcome',  # X-axis label
    yaxis_title='Counts',  # Y-axis label
    width=800,   # Width of the plot
    height=400,  # Height of the plot
)
fig_qhed.show()  # Display the figure

### ***Explanation of the Code Procedure:***


1- Importing necessary libraries:

- plotly.graph_objects: Used for creating a variety of interactive plots, including basic charts, complex statistical or financial plots, and 3D plots.
- qiskit: A Python library for simulating quantum circuits and running them on real quantum computers.-
- numpy: Used for numerical computations in Python.
- PIL (Python Imaging Library): Used for opening, manipulating, and saving many different image file formats.


2- Defining a function to process the image:

- The process_image function is defined to convert an image into grayscale and transform it into a numpy array.
- Inputs: image_path (the path of the image to process)
- The function opens the image, converts it to grayscale, and then converts the image data to a numpy array.
- Returns the image data and the size of the image.


3- Asking the user for the image path and processing the image:

- The user is asked to input the path of the image they want to process.
- The image is then processed using the process_image function defined earlier.
- The size of the original image is printed out.


4- Asking the user if they want to resize the image:

- The user is asked if they want to resize the image.
- If they choose to resize, they are asked to input the new dimensions, and the image is resized accordingly.
- If they don't want to resize the image, the smallest dimension of the original image is used as the size for the quantum image processing algorithms.


5- Defining the FRQI, NEQR, and QHED functions:

- These functions define the quantum circuits for the Fast Quantum Image Representation (FRQI), Novel Enhanced Quantum Representation (NEQR), and Quantum Hadamard Edge Detection (QHED) algorithms.
- Inputs: n (the size of the image) and image_data (the image data as a numpy array)
- Each function creates a quantum circuit, prepares the initial state, sets the grayscale value for each pixel, applies the necessary quantum gates, and measures the output.


6- Creating FRQI, NEQR, and QHED circuits:

- The quantum circuits for the FRQI, NEQR, and QHED algorithms are created using the image data.
- The image data is flattened into a one-dimensional array, and the length of this array is used as the size for the quantum circuits.


7- Defining a function to create a noise model with a specified error rate:

- The get_noise function takes a probability of error as input and creates a depolarizing error with this probability.
- A noise model is created to simulate the effects of noise in a real quantum computer.
- The depolarizing error is added to the noise model.
- A noise model with a 1% error rate is then created.

8- Defining a function to execute the circuit and get the results:

- The execute_circuit function is defined to execute a quantum circuit on a quantum simulator and return the results.
- Inputs: circuit (the quantum circuit to execute), noise_model (optional: a noise model to simulate the effects of noise), error_correction (optional: a flag indicating whether error correction should be applied).
- The function uses the Qiskit execute function to run the quantum circuit on the Qiskit qasm simulator and then gets the counts of each measurement outcome.


9- Executing the circuits in different scenarios:

- The FRQI, NEQR, and QHED circuits are executed both without noise and with the noise model.
- This is done to compare the results of the quantum image processing algorithms in ideal conditions and in conditions that simulate a real quantum computer.
- The counts of each measurement outcome are stored for each algorithm and for both the noiseless and noisy cases.


10- Plotting the results for FRQI, NEQR, and QHED:

- The results of the quantum image processing algorithms are plotted using Plotly.
- The results are shown as bar graphs, with the x-axis representing the measurement outcomes and the y-axis representing the counts of each outcome.
- Separate plots are made for the FRQI, NEQR, and QHED algorithms.
- For each algorithm, two bars are plotted for each measurement outcome: one for the noiseless case and one for the noisy case.
- The opacity of the bars is adjusted to make the plot easier to read.


11- Visualizing the plots:

- The plots for FRQI, NEQR, and QHED are displayed using the Plotly library.
- Each plot shows the results of the quantum image processing algorithms in different scenarios: noiseless, noisy, and noisy with error correction.

# **What is Barrier in Quantum Computing ?**

In quantum computing, a barrier is a directive for circuit compilation and optimization processes. It instructs the compiler that optimizations or reordering of operations (gates) should not be done across the barrier.

In Qiskit, *qc.barrier()* is used to add a barrier to all qubits in the circuit qc.

In terms of visualization, barriers are often used to make quantum circuits more readable. They provide a way of grouping operations visually. This can be especially helpful in large complex circuits where you want to highlight or isolate certain sets of operations.

Note that, in practice, barriers can affect the performance of your quantum circuit on actual quantum hardware. Quantum hardware has constraints and characteristics such as gate times and coherence times. If a barrier is placed in such a way that it forces a delay in executing the next operation, and this delay is longer than the coherence time, it could lead to increased quantum noise and decreased performance. This is an advanced topic in quantum computing and usually, one does not need to worry about it when just getting started.

To summarize, the main uses of barriers are:

    Directing circuit optimization processes during compilation.
    Improving the readability of the circuit diagram by visually separating groups of gates.